# DAT210x - Programming with Python for DS

## Module5- Lab10

In [1]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.utils.validation import check_random_state
import scipy.io.wavfile as wavfile

Good Luck! Heh.

### About Audio

Samples are Observations. Each audio file will is a single sample in our dataset.

Find more information about [Audio Samples here](https://en.wikipedia.org/wiki/Sampling_(signal_processing)).

Each .wav file is actually just a bunch of numeric samples, "sampled" from the analog signal. Sampling is a type of discretization. When we mention 'samples', we mean observations. When we mention 'audio samples', we mean the actually "features" of the audio file.

The goal of this lab is to use multi-target, linear regression to generate by extrapolation, the missing portion of the test audio file.

Each one audio_sample features will be the output of an equation, which is a function of the provided portion of the audio_samples:

    missing_samples = f(provided_samples)

You can experiment with how much of the audio you want to chop off and have the computer generate using the Provided_Portion parameter.

Play with this. This is how much of the audio file will be provided, in percent. The remaining percent of the file will be generated via linear extrapolation.

In [2]:
Provided_Portion = 0.25

### The Assignment

You have to download the dataset (audio files) from the website: https://github.com/Jakobovski/free-spoken-digit-dataset

Start by creating a regular Python List called `zero`:

Loop through the dataset and load up all 50 of the `0_jackson*.wav` files using the `wavfile.read()` method: https://docs.scipy.org/doc/scipy-0.14.0/reference/generated/scipy.io.wavfile.read.html Be careful! `.read()` returns a tuple and you're only interested in the audio data, and not sample_rate at this point. Inside your for loop, simply append the loaded audio data into your Python list `zero`:

In [3]:
zero = []

import os
for file in os.listdir('Datasets/free-spoken-digit-dataset-master/recordings'):
    if file.startswith('0_jackson'):
        a = os.path.join('Datasets/free-spoken-digit-dataset-master/recordings', file)
        sample_rate, audio_data = wavfile.read(a)
        zero.append(audio_data)
print (len(zero)) # 50, as expected since there 50 files in the folder starting with "0_jackson"



50


Just for a second, convert zero into a DataFrame. When you do so, set the `dtype` to `np.int16`, since the input audio files are 16 bits per sample. If you don't know how to do this, read up on the docs here: http://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.html

Since these audio clips are unfortunately not length-normalized, we're going to have to just hard chop them to all be the same length. Since Pandas would have inserted NANs at any spot to make zero a  perfectly rectangular [n_observed_samples, n_audio_samples] array, do a `dropna` on the Y axis here. Then, convert one back into an NDArray using `yourarrayname.values`:

In [4]:
zero = pd.DataFrame(data = zero, dtype = np.int16)
zero.dropna(axis = 1, inplace = True)
print (zero)
zero = zero.values
print (type(zero)) # zero is now a numpy NDArray
print (zero)

    0     1     2     3     4     5     6     7     8     9     ...   4077  \
0   -369  -431  -475  -543  -571  -557  -528  -455  -394  -305  ...   2475   
1   -311   -91  -140  -182  -271   -68  -235  -359  -129  -198  ...   -122   
2   -314  -303  -332  -376  -344  -344  -334  -295  -243  -224  ...  -2084   
3    347   351   462   451   499   509   469   534   362   464  ...   1163   
4   -336   160    65  -161   -23    61    39  -332   348  -198  ...    105   
5    354   442   610   728   768   842   873   784   739   696  ...    857   
6    397   531   638   699   744   769   732   694   601   508  ...   2162   
7    382   459   530   591   641   672   690   630   592   471  ...   -501   
8   -393    54   -71  -370   -75    39  -234  -308    79  -155  ...     21   
9   -311  -363  -318  -476  -474  -500  -452  -393  -400  -329  ...   -327   
10  -316  -336  -342  -351  -358  -369  -357  -331  -302  -279  ...  -1365   
11   335   392   481   519   597   604   592   623   501   473  

It's important to know how (many audio_samples samples) long the data is now.

`zero` is currently shaped like `[n_samples, n_audio_samples]`, so get the `n_audio_samples` count and store it in a variable called `n_audio_samples`:

In [17]:
n_audio_samples = zero.shape[1]
print (n_audio_samples) #4087


4087


Create your linear regression model here and store it in a variable called `model`. Don't actually train or do anything else with it yet:

In [18]:
from sklearn import linear_model
model = linear_model.LinearRegression()

There are 50 takes of each clip. You want to pull out just one of them, randomly, and that one will NOT be used in the training of your model. In other words, the one file we'll be testing / scoring on will be an unseen sample, independent to the rest of your training set:

In [7]:
# Leave this line alone until you've submitted your lab:
from sklearn.utils.validation import check_random_state
rng = check_random_state(7)

random_idx = rng.randint(zero.shape[0])
test  = zero[random_idx]
train = np.delete(zero, [random_idx], axis=0)

Print out the shape of `train`, and the shape of `test`.

`train` will be shaped: `[n_samples, n_audio_samples]`, where `n_audio_samples` are the 'features' of the audio file 

`test` will be shaped `[n_audio_features]`, since it is a single sample (audio file, e.g. observation).

In [19]:
print ("Shapes of train and test, respectively:", train.shape, test.shape)

Shapes of train and test, respectively: (49, 4087) (4087,)


The test data will have two parts, `X_test` and `y_test`.

`X_test` is going to be the first portion of the test audio file, which we will be providing the computer as input. 

`y_test`, the "label" if you will, is going to be the remaining portion of the audio file. Like such, the computer will use linear regression to derive the missing portion of the sound file based off of the training data its received!

Let's save the original `test` clip, the one you're about to delete half of, to the current directory so that you can compare it to the 'patched' clip once you've generated it. You should have already got the `sample_rate` when you were loading up the .wav files:

In [20]:
wavfile.write('Original Test Clip.wav', sample_rate, test)

Prepare the TEST data by creating a slice called `X_test`. It should have `Provided_Portion` * `n_audio_samples` audio sample features, taken from your test audio file, currently stored in variable `test`. In other words, grab the FIRST `Provided_Portion` * `n_audio_samples` audio features from `test` and store it in `X_test`. This should be accomplished using indexing:

In [25]:
X_test = test[:int(Provided_Portion*n_audio_samples)]
X_test

array([ -417,   152,   168, ..., -4055, -6846, -6992], dtype=int16)

If the first `Provided_Portion` * `n_audio_samples` features were stored in `X_test`, then we need to also grab the _remaining_ audio features and store them in `y_test`. With the remaining features stored in there, we will be able to R^2 "score" how well our algorithm did in completing the sound file.

In [26]:
y_test = test[int(Provided_Portion*n_audio_samples):]
y_test

array([-7992, -8650, -9005, ...,   638,   463,   332], dtype=int16)

Duplicate the same process for `X_train`, `y_train`. The only differences being:

1. Your will be getting your audio data from `train` instead of from `test`
2. Remember the shape of `train` that you printed out earlier? You want to do this slicing but for ALL samples (observations). For each observation, you want to slice the first `Provided_Portion` * `n_audio_samples` audio features into `X_train`, and the remaining go into `y_test`. All of this should be doable using regular indexing in two lines of code:

In [24]:
X_train = train[:, :int(Provided_Portion*n_audio_samples)]
y_train = train[:, int(Provided_Portion*n_audio_samples):]
X_train

array([[ -369,  -431,  -475, ...,  -256,  -240, -1102],
       [ -311,   -91,  -140, ...,   864,  1798,  3052],
       [ -314,  -303,  -332, ...,  -193,   487,  -294],
       ..., 
       [  364,   420,   469, ...,  -650, -1770, -2556],
       [  330,   382,   389, ...,   993,   677,   584],
       [ -312,  -335,  -338, ...,  3429,  5376,  6313]], dtype=int16)

SciKit-Learn gets 'angry' if you don't supply your training data in the form of a 2D dataframe shaped like `[n_samples, n_features]`.

So if you only have one SAMPLE, such as is our case with `X_test`, and `y_test`, then by calling `.reshape(1, -1)`, you can turn `[n_features]` into `[1, n_features]` in order to appease SciKit-Learn.

On the other hand, if you only have one FEATURE, you can alternatively call `.reshape(-1, 1)` on your data to turn `[n_samples]` into `[n_samples, 1]`.

Reshape X_test and y_test as [1, n_features]:

In [28]:
X_test = X_test.reshape(1, -1)
y_test = y_test.reshape(1, -1)
X_test                    #Check to see the amount of brackets that has increased by 1

array([[ -417,   152,   168, ..., -4055, -6846, -6992]], dtype=int16)

Fit your model using your training data and label:

In [29]:
model.fit(X_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

Use your model to predict the `label` of `X_test`. Store the resulting prediction in a variable called `y_test_prediction`:

In [30]:
y_test_prediction = model.predict(X_test)

SciKit-Learn will use float64 to generate your predictions so let's take those values back to int16, which is what our .wav files expect:

In [31]:
y_test_prediction = y_test_prediction.astype(dtype=np.int16)

Score how well your prediction would do for some good laughs, by passing in your test data and test label `y_test`:

In [33]:
score = model.score(X_test, y_test)
print ("Extrapolation R^2 Score: ", score)

Extrapolation R^2 Score:  0.0


Let's take the first `Provided_Portion` portion of the test clip, the part you fed into your linear regression model. Then, stitch that together with the 'abomination' the predictor model generated for you and then save the completed audio clip:

In [34]:
completed_clip = np.hstack((X_test, y_test_prediction))
wavfile.write('Extrapolated Clip.wav', sample_rate, completed_clip[0])

Congrats on making it to the end of this crazy lab and module =) !